In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 

application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Preprocessing data

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_count = application_df.APPLICATION_TYPE.value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_count[app_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_count[class_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
cat_list = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
cat_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encoded_df = pd.DataFrame(enc.fit_transform(application_df[cat_list]))

# Add the encoded variable names to the dataframe
encoded_df.columns = enc.get_feature_names(cat_list)
encoded_df.head()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encoded_df, left_index=True, right_index=True).drop(columns=cat_list, axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop("IS_SUCCESSFUL", axis=1)
y = application_df.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}hdf5"

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
mc = ModelCheckpoint(filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
804/804 [==============================] - 1s 888us/step - loss: 91773.8906 - accuracy: 0.4899
Epoch 2/100
804/804 [==============================] - 1s 855us/step - loss: 31009.8535 - accuracy: 0.4997
Epoch 3/100
804/804 [==============================] - 1s 883us/step - loss: 26124.1035 - accuracy: 0.4997
Epoch 4/100
804/804 [==============================] - 1s 882us/step - loss: 3171.3765 - accuracy: 0.4916
Epoch 5/100
763/804 [===========================>..] - ETA: 0s - loss: 1824.0138 - accuracy: 0.4967
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 873us/step - loss: 3554.5291 - accuracy: 0.4979
Epoch 6/100
804/804 [==============================] - 1s 891us/step - loss: 188.5017 - accuracy: 0.5320
Epoch 7/100
804/804 [==============================] - 1s 866us/step - loss: 0.7032 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 1s 858us/step - loss: 0.6935 - accuracy: 0.5321
Epoch 9/100
804

804/804 [==============================] - 1s 941us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 63/100
804/804 [==============================] - 1s 891us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 880us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 65/100
748/804 [==========================>...] - ETA: 0s - loss: 0.6910 - accuracy: 0.5331
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 887us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 894us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 898us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 879us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 70/100
775/804 [======================

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6345 - accuracy: 0.6279 - 274ms/epoch - 1ms/step
Loss: 0.634490966796875, Accuracy: 0.6278716921806335


### Second attempt

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 50)                1550      
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,551
Trainable params: 7,551
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
mc2 = ModelCheckpoint(filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)
# Train the model
fit_model2 = nn.fit(X_train, y_train, epochs=100, callbacks=[mc2])

Epoch 1/100
804/804 [==============================] - 1s 980us/step - loss: 0.6940 - accuracy: 0.5217
Epoch 2/100
804/804 [==============================] - 1s 955us/step - loss: 0.6929 - accuracy: 0.5240
Epoch 3/100
804/804 [==============================] - 1s 961us/step - loss: 0.6928 - accuracy: 0.5241
Epoch 4/100
804/804 [==============================] - 1s 977us/step - loss: 0.6923 - accuracy: 0.5238
Epoch 5/100
787/804 [============================>.] - ETA: 0s - loss: 0.6920 - accuracy: 0.5272
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 980us/step - loss: 0.6920 - accuracy: 0.5269
Epoch 6/100
804/804 [==============================] - 1s 966us/step - loss: 0.6919 - accuracy: 0.5276
Epoch 7/100
804/804 [==============================] - 1s 968us/step - loss: 0.6918 - accuracy: 0.5311
Epoch 8/100
804/804 [==============================] - 1s 952us/step - loss: 0.6919 - accuracy: 0.5241
Epoch 9/100
804/804 [=================

804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5304
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5289
Epoch 65/100
780/804 [============================>.] - ETA: 0s - loss: 0.6917 - accuracy: 0.5287
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5273
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5304
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5312
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5326
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5299
Epoch 70/100
804/804 [==============================] - ETA: 0s - loss: 0.6916 - accuracy: 0.5274
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 

### Third attempt

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                3520      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 10)                310       
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_add_layer/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_add_layer/weights.{epoch:02d}hdf5"

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Create a callback that saves the model's weights every 5 epochs
mc3 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model3 = nn.fit(X_train_scaled, y_train, epochs=150, callbacks=[mc3], verbose=1)

Epoch 1/150
804/804 [==============================] - 1s 948us/step - loss: 0.5690 - accuracy: 0.7230
Epoch 2/150
804/804 [==============================] - 1s 953us/step - loss: 0.5543 - accuracy: 0.7305
Epoch 3/150
804/804 [==============================] - 1s 953us/step - loss: 0.5523 - accuracy: 0.7316
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7325
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7326
Epoch 6/150
804/804 [==============================] - 1s 999us/step - loss: 0.5476 - accuracy: 0.7348
Epoch 7/150
804/804 [==============================] - 1s 961us/step - loss: 0.5473 - accuracy: 0.7349
Epoch 8/150
804/804 [==============================] - 1s 982us/step - loss: 0.5471 - accuracy: 0.7358
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7352
Epoch 10/150
804/804 [==============================] - 1s 991us/step - loss: 0

804/804 [==============================] - 1s 911us/step - loss: 0.5372 - accuracy: 0.7408
Epoch 80/150
804/804 [==============================] - 1s 964us/step - loss: 0.5365 - accuracy: 0.7408
Epoch 81/150
804/804 [==============================] - 1s 933us/step - loss: 0.5362 - accuracy: 0.7413
Epoch 82/150
804/804 [==============================] - 1s 998us/step - loss: 0.5364 - accuracy: 0.7414
Epoch 83/150
804/804 [==============================] - 1s 857us/step - loss: 0.5361 - accuracy: 0.7404
Epoch 84/150
804/804 [==============================] - 1s 877us/step - loss: 0.5362 - accuracy: 0.7407
Epoch 85/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7406
Epoch 86/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7409
Epoch 87/150
804/804 [==============================] - 1s 943us/step - loss: 0.5363 - accuracy: 0.7407
Epoch 88/150
804/804 [==============================] - 1s 972us/step - loss: 0.5

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7262 - 259ms/epoch - 967us/step
Loss: 0.5600535273551941, Accuracy: 0.7261807322502136


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")